# 实验三：序贯排序构建投资组合

## 数据处理

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv(
    '../../实验三/assets/data/course/RESSET_MRESSTK_1.csv', 
    encoding='gbk', usecols=range(8)
)
data 

,股票代码_Stkcd,日期_Date,收盘价_ClPr,总股数_Fullshr,流通股_Trdshr,月收益率_Monret,月无风险收益率_Monrfret,市盈率_PE
0,1,2000-01-28,18.53,1.551847e+09,1.071634e+09,0.0619,0.001650,51.90
1,1,2000-02-29,18.32,1.551847e+09,1.071634e+09,-0.0113,0.001650,51.32
2,1,2000-03-31,18.37,1.551847e+09,1.071634e+09,0.0027,0.001650,51.46
3,1,2000-04-28,19.05,1.551847e+09,1.071634e+09,0.0370,0.001650,56.53
4,1,2000-05-31,18.00,1.551847e+09,1.071634e+09,-0.0551,0.001650,53.41
...,...,...,...,...,...,...,...,...
49995,635,2017-07-31,20.22,3.030876e+08,3.030876e+08,0.1929,0.003596,41.43
49996,635,2017-08-31,20.71,3.030876e+08,3.030876e+08,0.0242,0.003602,42.44
49997,635,2017-09-29,17.57,3.030876e+08,3.030876e+08,-0.1516,0.003650,36.00
49998,635,2017-10-31,17.06,3.030876e+08,3.030876e+08,-0.0290,0.003641,29.84


In [3]:
for i in range(2, 10):
    data_read = pd.read_csv('../../实验三/assets/data/course/RESSET_MRESSTK_' + str(i) + '.csv', encoding='gbk', usecols=range(8))
    data = pd.concat((data, data_read), 
                     axis=0, # 行追加
                     ignore_index=True)
data.columns = ['stk', 'date', 'close', 'fshare', 'tshare', 'monret', 'monrf', 'pe']
data

,stk,date,close,fshare,tshare,monret,monrf,pe
0,1,2000-01-28,18.53,1.551847e+09,1.071634e+09,0.0619,0.001650,51.90
1,1,2000-02-29,18.32,1.551847e+09,1.071634e+09,-0.0113,0.001650,51.32
2,1,2000-03-31,18.37,1.551847e+09,1.071634e+09,0.0027,0.001650,51.46
3,1,2000-04-28,19.05,1.551847e+09,1.071634e+09,0.0370,0.001650,56.53
4,1,2000-05-31,18.00,1.551847e+09,1.071634e+09,-0.0551,0.001650,53.41
...,...,...,...,...,...,...,...,...
437756,603999,2019-08-30,5.62,5.760000e+08,5.760000e+08,0.0604,0.002218,61.96
437757,603999,2019-09-30,5.65,5.760000e+08,5.760000e+08,0.0053,0.002261,62.29
437758,603999,2019-10-31,5.86,5.760000e+08,5.760000e+08,0.0372,0.002304,54.56
437759,603999,2019-11-29,6.36,5.760000e+08,5.760000e+08,0.0853,0.002490,59.22


In [4]:
data['date'] = pd.to_datetime(data['date'])
data['yearmonth'] = data['date'].dt.strftime('%Y%m').astype(int)
# 计算股票市值(stksize) = 收盘价 × 流通股数
data['stksize'] = data['close']*data['fshare']
# 计算盈利收益率(stkep) = 1/市盈率，即E/P比率
data['stkep'] = 1/data['pe']
# 计算超额收益率(monexcret) = 月度收益率 - 无风险利率
data['monexcret'] = data['monret'] - data['monrf']
# 删除市值和盈利收益率为空的行
data.dropna(inplace = True, subset=['stksize', 'stkep'])

In [5]:
# 唯一的年月
uym = np.unique(data['yearmonth'].values)
print(len(uym))

240


In [6]:
print("总共有", len(uym), "个月份")
print("第一个月份:", uym[0], "最后一个月份:", uym[-1])
print("前10个月份:", uym[:10])

总共有 240 个月份
第一个月份: 200001 最后一个月份: 201912
前10个月份: [200001 200002 200003 200004 200005 200006 200007 200008 200009 200010]


一共有240个月份，接下来进行序贯排序

## 序贯排序操作

In [7]:
# 选择要分析的月份范围（比如第6个月开始的13个月）
selected_months = uym[5:5+13]
print(selected_months)

[200006 200007 200008 200009 200010 200011 200012 200101 200102 200103
 200104 200105 200106]


In [8]:
# 提取特定月份的数据

# 从基准月（第一个月）获取股票代码、市值和盈利收益率
dm = data.loc[data['yearmonth'] == selected_months[0], ['stk', 'stksize', 'stkep']]
dm.dropna(inplace=True)

# 依次从后续月份获取超额收益率，并与基准月数据合并
for i in range(1, len(selected_months)):
    ind = data['yearmonth'] == selected_months[i]
    # 获取当月的股票代码和超额收益率
    month_data = data.loc[ind, ['stk', 'monexcret']]
    # 与基准月数据合并
    dm = pd.merge(
        left=dm,
        right=month_data,
        on='stk',
        how='left',
        sort=True,
        suffixes=('', '_'+str(i))
    )

# 重命名列名
dm.columns = ['stk', 'size6', 'ep6', 'ret7', 'ret8', 'ret9', 'ret10', 'ret11', 
              'ret12', 'retn1', 'retn2', 'retn3', 'retn4', 'retn5', 'retn6']
dm

,stk,size6,ep6,ret7,ret8,ret9,ret10,ret11,ret12,retn1,retn2,retn3,retn4,retn5,retn6
0,1,2.813499e+10,0.018587,0.01935,-0.04275,-0.04615,0.03315,0.00775,-0.06425,0.03005,-0.06105,0.14955,-0.04415,0.04145,-0.05655
1,2,6.879519e+09,0.032031,0.02575,-0.04995,-0.05445,0.06545,0.05895,-0.00235,0.06555,-0.07935,0.09715,-0.03805,-0.02775,0.04135
2,3,1.609612e+09,-0.108108,0.08045,0.05045,-0.06385,0.08285,0.09295,-0.04865,-0.00835,-0.19095,0.12255,-0.08265,-0.23245,-0.01625
3,4,1.828172e+09,0.005067,0.26015,0.16905,0.00515,-0.15885,0.17025,0.03025,-0.15315,-0.10875,0.14155,0.00845,-0.00475,0.00665
4,5,5.371841e+09,0.013839,-0.03735,-0.00805,-0.15815,0.05755,0.05055,0.00015,0.11605,-0.09545,0.11205,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,600896,3.950168e+09,0.016287,-0.07075,-0.00075,-0.16195,0.05685,0.00905,0.04825,-0.01265,-0.06725,0.04785,-0.03555,0.07155,-0.00165
916,600897,3.645000e+09,0.014004,0.05765,-0.09395,-0.08335,0.06885,0.08455,0.02215,0.04765,-0.09505,0.07985,-0.04895,0.06885,0.02315
917,600898,9.760557e+08,-0.740741,0.26755,0.07175,NaN,NaN,NaN,NaN,0.28805,-0.37725,0.00945,NaN,NaN,NaN
918,600899,3.008716e+09,0.013569,-0.08715,0.08865,-0.01405,-0.05555,0.04205,0.02995,-0.04285,-0.03845,0.00795,0.56235,-0.20295,-0.01625


In [9]:
# 创建单因子排序标签

# 计算基准月股票总数
L = len(dm)
# 分组数量
gnum = 5
# 计算每组的股票数量（向下取整）
n = np.fix(L/gnum).astype(int)

# 例如，如果有1000只股票，每组应该包含 1000÷5 = 200 只股票。

# 创建单因子排序标签
# 创建一个长度为 L 的数组，所有元素初始值为1
x = np.ones(L)

i = 0
while i < gnum:
    if i == gnum-1: # 最后一组
        x[i*n:] = x[i*n:]*i  # 最后一组可能包含余数股票
    else:
        x[i*n:(i+1)*n] = x[i*n:(i+1)*n]*i  # 其他组平均分配
    i = i+1
'''
第0组股票（索引0到n-1）被赋值为 0（因为 1×0 = 0）
第1组股票（索引n到2n-1）被赋值为 1（因为 1×1 = 1）
第2组股票（索引2n到3n-1）被赋值为 2（因为 1×2 = 2）
第3组股票（索引3n到4n-1）被赋值为 3（因为 1×3 = 3）
第4组股票（索引4n到末尾）被赋值为 4（因为 1×4 = 4）
'''

# 将标签数组转换为整数类型，得到最终的单因子排序标签
ssi = x.astype(int)

ssi

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [10]:
# 创建双因子排序标签

# 计算一级分组（大组）的股票数量：每个市值组(大组)包含的股票数量，约为 L/gnum
l = np.fix(L/gnum).astype(int)
# 计算二级分组（小组）的股票数量：每个市值-EP组合(小组)包含的股票数量，约为 L/(gnum²)
n = np.fix(L/(gnum**2)).astype(int)

# 创建双因子排序标签
x = np.ones(L)
i = 0
while i < gnum:  # 外层循环处理市值组
    j = 0
    while j < gnum: # 内层循环处理EP组
        if j == gnum-1:
            if i == gnum-1:
                x[(i*l+j*n):] = x[(i*l+j*n):]*j
            else:
                x[(i*l+j*n):((i+1)*l)] = x[(i*l+j*n):((i+1)*l)]*j
        else:
            x[(i*l+j*n):(i*l+(j+1)*n)] = x[(i*l+j*n):(i*l+(j+1)*n)]*j
        j=j+1
    i=i+1
'''
外层循环i遍历5个市值组（0到4）
内层循环j在每个市值组内再划分5个EP组（0到4）
对于每个市值组内的EP分组，有三种情况：

常规情况：非最后一个EP组，简单分配j值
特殊情况1：非最后市值组的最后一个EP组，确保填充到市值组边界
特殊情况2：最后市值组的最后一个EP组，处理所有剩余的股票
'''
sdi = x.astype(int)

sdi

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,

这里的结构类似于：

```
[0,0,...,0, 1,1,...,1, 2,2,...,2, 3,3,...,3, 4,4,...,4,  # 市值组0内的EP分组
 0,0,...,0, 1,1,...,1, 2,2,...,2, 3,3,...,3, 4,4,...,4,  # 市值组1内的EP分组
 ...
 0,0,...,0, 1,1,...,1, 2,2,...,2, 3,3,...,3, 4,4,...,4]  # 市值组4内的EP分组
```

In [11]:
# 执行序贯排序

# 首先按市值从小到大排序
dm.sort_values(by=['size6'], ascending=True, inplace=True)
# 添加单因子排序标签，前20%的股票（市值最小的）会被标记为0，接下来20%的股票标记为1，依此类推。
dm['sinsort'] = ssi

# 然后按单因子排序索引和EP值从小到大排序
# 市值组0（最小市值）的股票，按EP值从小到大排列
# 市值组1的股票，按EP值从小到大排列
# 以此类推...
dm.sort_values(by=['sinsort', 'ep6'], ascending=[True, True], inplace=True)
# 添加双因子排序标签
dm['dousort'] = sdi
dm

,stk,size6,ep6,ret7,ret8,ret9,ret10,ret11,ret12,retn1,retn2,retn3,retn4,retn5,retn6,sinsort,dousort
860,600837,2.368391e+08,-1.612903,0.21325,0.13215,0.00435,0.15545,0.21485,0.11415,-0.13965,-0.44365,0.20365,0.10095,NaN,NaN,0,0
917,600898,9.760557e+08,-0.740741,0.26755,0.07175,NaN,NaN,NaN,NaN,0.28805,-0.37725,0.00945,NaN,NaN,NaN,0,0
137,556,1.400283e+09,-0.206186,0.19015,0.04305,-0.04305,0.08465,0.00795,-0.08175,-0.04295,-0.23855,0.07095,-0.05245,-0.07095,0.10265,0,0
14,15,5.130653e+08,-0.166113,0.15665,0.21245,-0.00725,0.15685,0.21495,0.13485,-0.15945,-0.37225,0.07835,0.15675,0.15285,NaN,0,0
659,600625,5.814446e+08,-0.162866,0.17875,0.19545,-0.09395,0.15085,0.05715,0.08165,0.03045,-0.21035,-0.19315,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,600068,5.980800e+09,0.048709,0.08685,-0.03035,-0.11925,0.01575,0.04105,-0.00045,0.07325,0.01245,0.02735,-0.07675,-0.01295,0.04735,4,4
195,629,7.244965e+09,0.049579,0.15795,-0.07655,-0.09885,0.09255,0.02025,-0.03105,0.11265,0.01815,0.06865,-0.08545,-0.02515,0.04515,4,4
314,800,9.960300e+09,0.054585,0.18625,-0.09505,-0.09395,-0.00675,0.09715,0.00145,0.08955,-0.02285,0.07935,-0.06595,-0.00025,-0.02875,4,4
254,709,1.034326e+10,0.055249,0.08345,-0.02455,-0.06585,0.00895,0.06885,-0.02605,0.03335,0.02005,0.05505,-0.06205,-0.01715,0.03945,4,4


In [12]:
# 计算各投资组合的平均收益率

# 计算第一个月（ret7）的分组平均收益率
spmreturn = dm.loc[:, ['ret7', 'sinsort', 'dousort']].dropna().groupby(  # 分组
            by=['sinsort', 'dousort']
            )['ret7'].mean() # 对每个分组计算'ret7'的平均值

# 依次计算后续月份的分组平均收益率
lret = ['ret8', 'ret9', 'ret10', 'ret11', 'ret12', 'retn1', 'retn2', 'retn3', 'retn4', 'retn5', 'retn6']
for i in lret:
    a = dm.loc[:, [i, 'sinsort', 'dousort']].dropna().groupby(
        by=['sinsort', 'dousort'])[i].mean()
    spmreturn = pd.concat([spmreturn, a], axis=1) # 扩展列合并

spmreturn

ret7      ret8      ret9     ret10     ret11     ret12  \
sinsort dousort                                                               
0       0        0.173922  0.107881  0.030353  0.060070  0.048401 -0.034741   
        1        0.111039  0.042821  0.003799  0.077228  0.066200 -0.018750   
        2        0.052022  0.056031 -0.019269  0.057953  0.078311  0.012172   
        3        0.028703  0.024397 -0.012497  0.049692  0.050439  0.029194   
        4        0.046168  0.008435 -0.009587  0.050182  0.076828  0.042158   
1       0        0.063858  0.056269 -0.050950  0.056687  0.063124  0.016793   
        1        0.041611  0.014114 -0.012194  0.032910  0.076027  0.013736   
        2        0.049258  0.001844 -0.027422  0.070122  0.053942  0.005214   
        3        0.054975  0.008581 -0.013614  0.045028  0.094844  0.016283   
        4        0.066050 -0.016965 -0.033098  0.065848  0.062397  0.015870   
2       0        0.057742  0.042617 -0.037903  0.050064  0.076611 -0.018497   
        1        0.014333 -0.010058 -0.041322  0.020497  0.040197  0.020739   
        2        0.056300  0.002936 -0.064081  0.038628  0.063406  0.001269   
        3        0.021228 -0.010472 -0.043994  0.027931  0.062028  0.010069   
        4        0.082727 -0.021792 -0.063348  0.026665  0.064165  0.019290   
3       0        0.016956 -0.019764 -0.036093  0.021831  0.045503 -0.002228   
        1        0.036478 -0.002578 -0.034219  0.015856  0.053086  0.008631   
        2        0.020203  0.011294 -0.039233  0.032125  0.048550  0.011708   
        3        0.051350 -0.015219 -0.056600  0.028619  0.066547  0.010719   
        4        0.062085 -0.028233 -0.070782  0.027050  0.068445  0.013453   
4       0        0.028142 -0.003422 -0.046336 -0.003061  0.051483 -0.014019   
        1        0.016983 -0.023992 -0.066517 -0.000075  0.064961 -0.030997   
        2        0.041000  0.002325 -0.035553  0.001619  0.048522 -0.011456   
        3        0.027756 -0.020106 -0.067797  0.012744  0.041869  0.001794   
        4        0.059242 -0.024403 -0.081827  0.013978  0.071618  0.000600   

                    retn1     retn2     retn3     retn4     retn5     retn6  
sinsort dousort                                                              
0       0       -0.017000 -0.175225  0.116478  0.043306  0.064793  0.045944  
        1       -0.046994 -0.088456  0.112203  0.046222  0.115211 -0.022586  
        2       -0.036836 -0.065128  0.087808  0.044842  0.093056 -0.003072  
        3       -0.031914 -0.072972  0.113317  0.018290  0.082219 -0.013841  
        4       -0.015373 -0.058518  0.090845  0.002165  0.091280  0.017025  
1       0       -0.037836 -0.104803  0.132569  0.011724  0.100701  0.000453  
        1       -0.018678 -0.059683  0.080008  0.014997  0.081139  0.000600  
        2        0.002089 -0.069831  0.077628  0.006139  0.068942  0.025783  
        3       -0.000567 -0.055572  0.104931 -0.002844  0.067519  0.005322  
        4        0.004270 -0.035415  0.085587  0.003575  0.097543 -0.001230  
2       0       -0.019614 -0.083525  0.099369  0.006139  0.062622 -0.003950  
        1       -0.023200 -0.097872  0.076550  0.006814  0.042106  0.056844  
        2        0.002392 -0.053286  0.056639  0.019853  0.045672  0.016058  
        3       -0.011833 -0.066153  0.087167 -0.013842  0.053353  0.019775  
        4        0.018185 -0.038035  0.057118 -0.018145  0.034603  0.017068  
3       0       -0.005361 -0.074056  0.070358  0.020567  0.057011 -0.027172  
        1       -0.010447 -0.051083  0.077078 -0.002583  0.022444 -0.018972  
        2       -0.008208 -0.062942  0.067153 -0.028097  0.039258  0.009069  
        3        0.001817 -0.056867  0.077853 -0.014053  0.016761  0.010750  
        4        0.012778 -0.040185  0.066818 -0.031177  0.014087  0.016700  
4       0       -0.054681 -0.078636  0.042200 -0.008781  0.030781 -0.011658  
        1       -0.026714 -0.078544  0.069389 -0.001327  0.002239 -0.005936  
        2      

In [13]:
# 计算各组合在所有月份的平均收益率
spmreturn['mret'] = spmreturn.apply(lambda x: x.mean(), axis=1)
spmreturn

ret7      ret8      ret9     ret10     ret11     ret12  \
sinsort dousort                                                               
0       0        0.173922  0.107881  0.030353  0.060070  0.048401 -0.034741   
        1        0.111039  0.042821  0.003799  0.077228  0.066200 -0.018750   
        2        0.052022  0.056031 -0.019269  0.057953  0.078311  0.012172   
        3        0.028703  0.024397 -0.012497  0.049692  0.050439  0.029194   
        4        0.046168  0.008435 -0.009587  0.050182  0.076828  0.042158   
1       0        0.063858  0.056269 -0.050950  0.056687  0.063124  0.016793   
        1        0.041611  0.014114 -0.012194  0.032910  0.076027  0.013736   
        2        0.049258  0.001844 -0.027422  0.070122  0.053942  0.005214   
        3        0.054975  0.008581 -0.013614  0.045028  0.094844  0.016283   
        4        0.066050 -0.016965 -0.033098  0.065848  0.062397  0.015870   
2       0        0.057742  0.042617 -0.037903  0.050064  0.076611 -0.018497   
        1        0.014333 -0.010058 -0.041322  0.020497  0.040197  0.020739   
        2        0.056300  0.002936 -0.064081  0.038628  0.063406  0.001269   
        3        0.021228 -0.010472 -0.043994  0.027931  0.062028  0.010069   
        4        0.082727 -0.021792 -0.063348  0.026665  0.064165  0.019290   
3       0        0.016956 -0.019764 -0.036093  0.021831  0.045503 -0.002228   
        1        0.036478 -0.002578 -0.034219  0.015856  0.053086  0.008631   
        2        0.020203  0.011294 -0.039233  0.032125  0.048550  0.011708   
        3        0.051350 -0.015219 -0.056600  0.028619  0.066547  0.010719   
        4        0.062085 -0.028233 -0.070782  0.027050  0.068445  0.013453   
4       0        0.028142 -0.003422 -0.046336 -0.003061  0.051483 -0.014019   
        1        0.016983 -0.023992 -0.066517 -0.000075  0.064961 -0.030997   
        2        0.041000  0.002325 -0.035553  0.001619  0.048522 -0.011456   
        3        0.027756 -0.020106 -0.067797  0.012744  0.041869  0.001794   
        4        0.059242 -0.024403 -0.081827  0.013978  0.071618  0.000600   

                    retn1     retn2     retn3     retn4     retn5     retn6  \
sinsort dousort                                                               
0       0       -0.017000 -0.175225  0.116478  0.043306  0.064793  0.045944   
        1       -0.046994 -0.088456  0.112203  0.046222  0.115211 -0.022586   
        2       -0.036836 -0.065128  0.087808  0.044842  0.093056 -0.003072   
        3       -0.031914 -0.072972  0.113317  0.018290  0.082219 -0.013841   
        4       -0.015373 -0.058518  0.090845  0.002165  0.091280  0.017025   
1       0       -0.037836 -0.104803  0.132569  0.011724  0.100701  0.000453   
        1       -0.018678 -0.059683  0.080008  0.014997  0.081139  0.000600   
        2        0.002089 -0.069831  0.077628  0.006139  0.068942  0.025783   
        3       -0.000567 -0.055572  0.104931 -0.002844  0.067519  0.005322   
        4        0.004270 -0.035415  0.085587  0.003575  0.097543 -0.001230   
2       0       -0.019614 -0.083525  0.099369  0.006139  0.062622 -0.003950   
        1       -0.023200 -0.097872  0.076550  0.006814  0.042106  0.056844   
        2        0.002392 -0.053286  0.056639  0.019853  0.045672  0.016058   
        3       -0.011833 -0.066153  0.087167 -0.013842  0.053353  0.019775   
        4        0.018185 -0.038035  0.057118 -0.018145  0.034603  0.017068   
3       0       -0.005361 -0.074056  0.070358  0.020567  0.057011 -0.027172   
        1       -0.010447 -0.051083  0.077078 -0.002583  0.022444 -0.018972   
        2       -0.008208 -0.062942  0.067153 -0.028097  0.039258  0.009069   
        3        0.001817 -0.056867  0.077853 -0.014053  0.016761  0.010750   
        4        0.012778 -0.040185  0.066818 -0.031177  0.014087  0.016700   
4       0       -0.054681 -0.078636  0.042200 -0.008781  0.030781 -0.011658   
        1       -0.026714 -0.078544  0.069389 -0.001327  0.002239 -0.0

In [14]:
# 初始化结果存储
meanret = []
lcname = []

# 从第6个月开始，每隔12个月选择一个新的起始月份
for i in range(5, 234, 12):
    # 确保有足够的月份
    if len(uym[i:i+13]) == 13:
        # 保存基准月作为列名
        lcname.append(str(uym[i]))
        
        # 重新执行序贯排序过程
        # 1. 提取数据
        months = uym[i:i+13]
        dm = data.loc[data['yearmonth'] == months[0], ['stk', 'stksize', 'stkep']]
        dm.dropna(inplace=True)
        
        # 2. 合并后续月份数据
        for j in range(1, len(months)):
            ind = data['yearmonth'] == months[j]
            month_data = data.loc[ind, ['stk', 'monexcret']]
            dm = pd.merge(
                left=dm,
                right=month_data,
                on='stk',
                how='left',
                sort=True,
                suffixes=('', '_'+str(j))
            )
        
        # 3. 重命名列
        dm.columns = ['stk', 'size6', 'ep6', 'ret7', 'ret8', 'ret9', 'ret10', 'ret11', 
                      'ret12', 'retn1', 'retn2', 'retn3', 'retn4', 'retn5', 'retn6']
        
        # 4. 执行排序
        dm.sort_values(by=['size6'], ascending=True, inplace=True)
        
        # 5. 创建单因子排序标签（确保范围为0-4）
        L = len(dm)
        n = np.fix(L/gnum).astype(int)
        
        # 清晰地创建单因子排序标签
        ssi = np.zeros(L, dtype=int)
        for g in range(gnum):
            start_idx = g * n
            end_idx = (g + 1) * n if g < gnum - 1 else L
            ssi[start_idx:end_idx] = g
        
        dm['sinsort'] = ssi
        
        # 6. 在每个市值组内按EP值排序
        result_dfs = []
        for g in range(gnum):
            # 获取当前市值组的数据
            group_data = dm[dm['sinsort'] == g].copy()
            # 在组内按EP值排序
            group_data.sort_values(by=['ep6'], ascending=True, inplace=True)
            
            # 为组内数据创建EP组标签
            L_group = len(group_data)
            n_group = np.fix(L_group/gnum).astype(int)
            
            dousort = np.zeros(L_group, dtype=int)
            for ep_g in range(gnum):
                start_idx = ep_g * n_group
                end_idx = (ep_g + 1) * n_group if ep_g < gnum - 1 else L_group
                dousort[start_idx:end_idx] = ep_g
            
            group_data['dousort'] = dousort
            result_dfs.append(group_data)
        
        # 合并所有市值组的结果
        dm = pd.concat(result_dfs)
        
        # 7. 创建所有可能的组合索引
        all_combinations = []
        for si in range(gnum):
            for di in range(gnum):
                all_combinations.append((si, di))
        
        # 8. 计算各组合收益率
        # 为第一个月创建结果DataFrame，确保包含所有组合
        first_month_returns = dm.loc[:, ['ret7', 'sinsort', 'dousort']].dropna().groupby(
                              by=['sinsort', 'dousort'])['ret7'].mean()
        
        # 创建多级索引DataFrame
        index = pd.MultiIndex.from_tuples(all_combinations, names=['sinsort', 'dousort'])
        spmreturn = pd.DataFrame(index=index)
        
        # 填充第一个月数据
        spmreturn['ret7'] = first_month_returns
        
        # 填充后续月份数据
        lret = ['ret8', 'ret9', 'ret10', 'ret11', 'ret12', 'retn1', 'retn2', 'retn3', 'retn4', 'retn5', 'retn6']
        for ret in lret:
            month_returns = dm.loc[:, [ret, 'sinsort', 'dousort']].dropna().groupby(
                           by=['sinsort', 'dousort'])[ret].mean()
            spmreturn[ret] = month_returns
        
        # 计算平均收益率
        spmreturn['mret'] = spmreturn.mean(axis=1, skipna=True)
        
        # 9. 保存结果
        if len(meanret) == 0:
            meanret = pd.DataFrame(spmreturn['mret'])
        else:
            meanret[str(months[0])] = spmreturn['mret']

# 设置列名
meanret.columns = lcname

meanret

200006    200106    200206    200306    200406    200506  \
sinsort dousort                                                               
0       0        0.038682 -0.005377 -0.024413 -0.035574 -0.050378  0.047140   
        1        0.033161 -0.023603 -0.025662 -0.022485 -0.038628  0.047678   
        2        0.029824 -0.022175 -0.024993 -0.022466 -0.036893  0.039064   
        3        0.022085 -0.017776 -0.022063 -0.019937 -0.036897  0.033315   
        4        0.028467 -0.019226 -0.024360 -0.014133 -0.030322  0.038342   
1       0        0.025716 -0.019017 -0.029728 -0.031086 -0.044553  0.045151   
        1        0.022049 -0.020558 -0.028603 -0.025905 -0.033245  0.043136   
        2        0.021976 -0.023676 -0.026548 -0.016883 -0.038696  0.039611   
        3        0.027074 -0.021176 -0.022270 -0.019754 -0.026772  0.034882   
        4        0.026203 -0.018704 -0.017574 -0.010794 -0.020504  0.047784   
2       0        0.019306 -0.022034 -0.033642 -0.022505 -0.042536  0.037039   
        1        0.008802 -0.024747 -0.027950 -0.016537 -0.035253  0.031819   
        2        0.015482 -0.022552 -0.024913 -0.017505 -0.030481  0.034622   
        3        0.011271 -0.022214 -0.023799 -0.015691 -0.025381  0.042323   
        4        0.014875 -0.018801 -0.016940 -0.007300 -0.017269  0.045837   
3       0        0.005629 -0.020335 -0.025626 -0.021973 -0.029998  0.039979   
        1        0.007807 -0.026038 -0.019046 -0.016388 -0.029749  0.043836   
        2        0.008407 -0.021592 -0.026175 -0.014844 -0.024822  0.044335   
        3        0.010140 -0.019286 -0.019134 -0.009874 -0.020916  0.044331   
        4        0.009253 -0.014207 -0.016382 -0.007245 -0.019954  0.030035   
4       0       -0.005666 -0.019738 -0.019139 -0.028176 -0.038957  0.030799   
        1       -0.006711 -0.022039 -0.021726 -0.018425 -0.029096  0.038966   
        2       -0.000065 -0.012921 -0.014361 -0.011464 -0.018946  0.033696   
        3       -0.002183 -0.014030 -0.011614  0.001093 -0.011531  0.033168   
        4        0.006690 -0.011178 -0.006434  0.002890 -0.012345  0.025247   

                   200606    200706    200806    200906    201006    201106  \
sinsort dousort                                                               
0       0        0.126461  0.016072  0.055135  0.011673  0.032716 -0.005958   
        1        0.107215  0.010052  0.052488  0.027131  0.035022 -0.015669   
        2        0.099709  0.004965  0.050275  0.017892  0.038306 -0.017326   
        3        0.084425  0.013387  0.056171  0.024127  0.041007 -0.014898   
        4        0.078141  0.039795  0.082930  0.033111  0.034234 -0.009028   
1       0        0.095170 -0.004208  0.049865  0.009414  0.031882 -0.029026   
        1        0.085578 -0.005073  0.040033  0.015151  0.024619 -0.023169   
        2        0.070726 -0.008115  0.044065  0.019681  0.023501 -0.015293   
        3        0.077731 -0.005292  0.046263  0.024720  0.024113 -0.018248   
        4        0.084115  0.000501  0.047937  0.013542  0.028363 -0.019084   
2       0        0.076972 -0.020181  0.040663  0.007471  0.022658 -0.032158   
        1        0.077054 -0.015766  0.038448  0.004174  0.017234 -0.022232   
        2        0.068204 -0.004877  0.038947  0.021823  0.023063 -0.019279   
        3        0.070009 -0.009578  0.041036  0.021652  0.025357 -0.020303   
        4        0.076582 -0.002288  0.043236  0.018815  0.026551 -0.023915   
3       0        0.083818 -0.019195  0.025172  0.001288  0.024731 -0.028191   
        1        0.072510 -0.027005  0.019757  0.010557  0.016512 -0.024181   
        2        0.074009 -0.013109  0.021640  0.005354  0.012652 -0.017984   
        3        0.074859 -0.014391  0.034825  0.018344  0.018339 -0.018699   
        4        0.090957 -0.008379  0.035240  0.007685  0.024232 -0.022785   
4       0        0.081013 -0.024208  0.019698 -0.004682  0.016555 -0.026459   
        1        0.074267 -0.021017  0.017838 -0.009655  0.017331 -0

In [ ]:
# 计算所有时期的平均收益率并格式化输出

meanret['meanreturn'] = meanret.apply(lambda x: x.mean(), axis=1)
a = meanret['meanreturn'].values.reshape((gnum, gnum))
a

array([[ 0.02006561,  0.01750831,  0.01614524,  0.01721494,  0.02288047],
       [ 0.00933385,  0.00969608,  0.00951656,  0.01102833,  0.013898  ],
       [ 0.00348249,  0.00484817,  0.00785716,  0.00797597,  0.01072822],
       [ 0.00290672,  0.00332861,  0.00412666,  0.00727395,  0.00916713],
       [-0.0004822 ,  0.00064517,  0.00330825,  0.00708807,  0.00853755]])

In [16]:
# 格式化打印结果
print('{:>10s} {:>10s}, {:>10s}, {:>10s}, {:>10s}, {:>10s}'.format('', 'EP1', 'EP2', 'EP3', 'EP4', 'EP5'))
for i in range(gnum):
    print('{:>10s} {:10.5f}, {:10.5f}, {:10.5f}, {:10.5f}, {:10.5f}'.format(
        'SIZE'+str(i+1), 
        a[i, 0], 
        a[i, 1], 
        a[i, 2], 
        a[i, 3], 
        a[i, 4]))

                  EP1,        EP2,        EP3,        EP4,        EP5
     SIZE1    0.02007,    0.01751,    0.01615,    0.01721,    0.02288
     SIZE2    0.00933,    0.00970,    0.00952,    0.01103,    0.01390
     SIZE3    0.00348,    0.00485,    0.00786,    0.00798,    0.01073
     SIZE4    0.00291,    0.00333,    0.00413,    0.00727,    0.00917
     SIZE5   -0.00048,    0.00065,    0.00331,    0.00709,    0.00854


## 数学原理解释

序贯排序构建投资组合是一种多因子投资策略，核心数学原理如下：

1. **双重排序公式**：
   
   假设有N只股票，分为G×G个组合，每个组合的平均超额收益率为：
   
   $$R_{i,j} = \frac{1}{N_{i,j}} \sum_{k \in S_{i,j}} (r_k - r_f)$$
   
   其中：
   - $R_{i,j}$ 是第i组市值和第j组EP值的平均超额收益率
   - $N_{i,j}$ 是该组合中的股票数量
   - $S_{i,j}$ 是该组合的股票集合
   - $r_k$ 是股票k的收益率
   - $r_f$ 是无风险利率

2. **投资组合构建逻辑**：
   
   首先按市值因子排序：
   $$Rank_{Size}(i) = \lfloor \frac{i \times G}{N} \rfloor, \quad i=0,1,...,N-1$$
   
   然后在每个市值组内按EP因子排序：
   $$Rank_{EP}(i,j) = \lfloor \frac{j \times G}{N_i} \rfloor, \quad j=0,1,...,N_i-1$$
   
   其中$N_i$是第i个市值组的股票数量。

3. **因子溢价计算**：
   
   规模溢价 = 小市值组合的平均收益率 - 大市值组合的平均收益率
   
   价值溢价 = 高EP组合的平均收益率 - 低EP组合的平均收益率

## 结果分析

从最终打印的结果可以观察到：

1. 市值最小的组合（SIZE1）平均收益率最高，市值最大的组合（SIZE5）平均收益率最低，体现了"小市值效应"
2. 在控制市值因子后，EP值越高的组合（EP5列）平均收益率一般越高，体现了"价值效应"
3. 小市值+高EP组合（SIZE1-EP5：0.02288）的平均收益率远高于大市值+低EP组合（SIZE5-EP1：-0.00043），说明结合两个因子可以获得更好的收益表现

这一研究结果支持了多因子投资理论中的规模效应和价值效应在中国股市的存在性，也为构建投资组合提供了理论依据。